<a href="https://colab.research.google.com/github/GitMishka/2021_TSLAstock_Analysis-/blob/master/paylocity1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas  matplotlib xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('/content/drive/MyDrive/Work/Paylocity Turnover/Book3.csv')


In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files

def extract_month(date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d')
    return date.month

# Read the spreadsheet into a Pandas DataFrame
# df = pd.read_csv('your_file.csv')

# Separate the data into different DataFrames based on LocationCode
location_codes = ['FRKT', 'FRKL', 'LEXN', 'LEXL', 'LEXE', 'LVLL', 'LVLM', 'FRLN']
dfs_by_location = {}
for location in location_codes:
    dfs_by_location[location] = df[df['LocationCode'] == location]

# Process each DataFrame separately
processed_dfs = {}
for location, df_location in dfs_by_location.items():
    df_location['TerminationDate'] = pd.to_datetime(df_location['TerminationDate'])
    df_location['TerminationMonth'] = df_location['TerminationDate'].apply(lambda x: x.strftime('%Y-%m'))
    total_count = df_location.groupby('TerminationMonth').size().reset_index(name='Total_count')
    ed_values = ['ASSISTANTED', 'ASSTED', 'CAMPUSED', 'ED', 'SENIORED']
    ed_count = df_location[df_location['PositionCode'].isin(ed_values)].groupby('TerminationMonth').size().reset_index(name='ED_count')
    bom_count = df_location[df_location['PositionCode'].str.contains('BOM', na=False)].groupby('TerminationMonth').size().reset_index(name='BOM_count')
    don_count = df_location[df_location['PositionCode'].str.contains('DON', na=False)].groupby('TerminationMonth').size().reset_index(name='DON_count')
    crd_count = df_location[df_location['PositionCode'].str.contains('CRD', na=False)].groupby('TerminationMonth').size().reset_index(name='CRD_count')
    processed_dfs[location] = total_count.merge(ed_count, on='TerminationMonth', how='left') \
        .merge(bom_count, on='TerminationMonth', how='left') \
        .merge(don_count, on='TerminationMonth', how='left') \
        .merge(crd_count, on='TerminationMonth', how='left')

# Save each processed DataFrame as a separate sheet in the same Excel file
with pd.ExcelWriter('location_data.xlsx') as writer:
    for location, processed_df in processed_dfs.items():
        processed_df.to_excel(writer, sheet_name=location, index=False)

# Download the Excel file
files.download('location_data.xlsx')


<ipython-input-10-b15470f67d7d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location['TerminationDate'] = pd.to_datetime(df_location['TerminationDate'])
<ipython-input-10-b15470f67d7d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location['TerminationMonth'] = df_location['TerminationDate'].apply(lambda x: x.strftime('%Y-%m'))
<ipython-input-10-b15470f67d7d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>